In [1210]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [1211]:
df_final2 = pd.read_csv("C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/CSV generated/df_final2.csv")
df_final2.columns = df_final2.columns.str.strip().str.replace(' ', '_')
df_final2

,submitter_id,tumor_descriptor,sample_type,tissue_type,ajcc_pathologic_stage,tissue_or_organ_of_origin,days_to_last_follow_up,age_at_diagnosis,primary_diagnosis,prior_malignancy,ajcc_staging_system_edition,ajcc_pathologic_t,morphology,ajcc_pathologic_n,ajcc_pathologic_m,treatment_or_therapy,ethnicity,gender,race,vital_status,age_at_index,days_to_death,days_to_birth,year_of_birth,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH10_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,DCC_exp,ERBB4_exp,FAM123B_exp,FAM47C_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met
0,TCGA-DC-6158,Primary,Primary Tumor,Tumor,Stage I,Rectum,216.0,25842.0,Adenocarcinoma,no,7th,T2,8140/3,N0,M0,no,not hispanic or latino,male,white,Dead,70.0,334.0,-25842.0,1940.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4924,0.9748,1.3970,0.6288,1.2007,-0.1100,2.0609,0.3974,0.5973,1.8436,1.1990,1.1869,0.0707,-0.0260,-0.6092,-0.7453,0.6173,1.1064,-0.1753,-1.2877,-0.9818,-0.6245,2.0391,-1.3237,-0.1122,2.3358,1.4414,-0.1442,0.0962,-0.4011,-1.1856,1.2024,0.2879,2.1324,-0.0907,0.2257,1.4222,1.6625,0.1280,-0.0253,1.2033,-0.0115,0.2442,-0.0368,-0.3021,0.0596,2.0379,0.5959,1.5926,0.0494,0.081050,0.062878,0.346054,0.319615,0.052065,0.064752,0.049225,0.180710,0.039378,0.029382,0.048825,0.477222,0.612599,0.482879,0.825678,0.409599,0.570405,0.065553,0.422390,0.227364,0.052323,0.059601,0.073225,0.249039,0.037806,0.137725,0.153706,0.034230,0.034982,0.064444,0.434121,0.061711,0.659091,0.109629,0.224915,0.433367,0.058690,0.176453,0.076784,0.125888,0.080165
1,TCGA-F4-6854,Primary,Primary Tumor,Tumor,Stage IIA,Sigmoid colon,16.0,28272.0,Adenocarcinoma,no,7th,T3,8140/3,N0,M0,no,not hispanic or latino,female,white,Alive,77.0,NaN,-28272.0,1934.0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.4356,0.7817,0.3864,-0.5794,0.0097,-0.3238,0.7983,0.3306,0.7006,0.5779,0.9375,0.6519,0.8877,-0.5103,-1.5957,-0.8619,0.7037,0.8379,-1.1649,0.0718,-0.3113,-1.2580,1.2076,-1.3237,2.0133,0.6225,0.6105,0.7847,-0.1232,-0.7184,-0.7204,1.1681,0.6379,-1.0807,1.6062,0.6152,0.3046,1.0671,-0.2328,-0.1504,1.1834,0.6032,-0.1104,0.4658,-0.5869,-0.7528,0.6788,1.1528,2.0313,0.5799,0.092701,0.058846,0.474341,0.149368,0.055344,0.216915,0.055520,0.174396,0.196454,0.033386,0.040905,0.472778,0.684621,0.911537,0.905876,0.677952,0.302979,0.422829,0.583546,0.472932,0.216605,0.053988,0.059024,0.424834,0.042549,0.644751,0.245062,0.036261,0.404158,0.066936,0.563126,0.268761,0.644221,0.103751,0.197726,0.283495,0.052923,0.111043,0.069776,0.108654,0.067182
2,TCGA-DC-5337,Primary,Primary Tumor,Tumor,Stage I,Rectosigmoid junction,792.0,25202.0,Adenocarcinoma,no,7th,T1,8140/3,N0,M0,no,not hispanic or latino,male,white,Alive,69.0,NaN,-25202.0,1941.0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.5716,-1.8757,-1.1666,-1.1679,-2.1978,-1.2844,-0.6873,-1.3785,-2.8494,-1.9309,-2.0135,

In [1212]:
dtype_df = pd.DataFrame(df_final2.dtypes, columns=['Data Type'])
#pd.set_option('display.max_rows', None)
dtype_df[:30]

,Data Type
submitter_id,object
tumor_descriptor,object
sample_type,object
tissue_type,object
ajcc_pathologic_stage,object
tissue_or_organ_of_origin,object
days_to_last_follow_up,float64
age_at_diagnosis,float64
primary_diagnosis,object
prior_malignancy,object


In [1213]:
print(df_final2.isna().sum())

submitter_id              0
tumor_descriptor          0
sample_type               0
tissue_type               0
ajcc_pathologic_stage    24
                         ..
SMAD4_met                42
TCF7L2_met               42
TNC_met                  42
TRRAP_met                42
ZFHX3_met                42
Length: 165, dtype: int64


In [1214]:
pd.set_option('display.max_columns', None)
print(df_final2)

     submitter_id tumor_descriptor           sample_type tissue_type  \
0    TCGA-DC-6158          Primary         Primary Tumor       Tumor   
1    TCGA-F4-6854          Primary         Primary Tumor       Tumor   
2    TCGA-DC-5337          Primary         Primary Tumor       Tumor   
3    TCGA-AA-3561          Primary         Primary Tumor       Tumor   
4    TCGA-AA-A00O          Primary         Primary Tumor       Tumor   
..            ...              ...                   ...         ...   
628  TCGA-AA-3818          Primary         Primary Tumor       Tumor   
629  TCGA-AG-A00C          Primary         Primary Tumor       Tumor   
630  TCGA-A6-2680   Not Applicable   Solid Tissue Normal      Normal   
631  TCGA-G5-6233   Not Applicable  Blood Derived Normal      Normal   
632  TCGA-DM-A28G   Not Applicable  Blood Derived Normal      Normal   

    ajcc_pathologic_stage tissue_or_organ_of_origin  days_to_last_follow_up  \
0                 Stage I                    Rectum     

## Data wrangling

### Non-relevant columns

Let's remove non-relevant columns:\
Submitter_id
Tumor, descritors, tissue types... all this is more in detail in TNM staging; variables with same info does not improve the model\
ajcc_staging_system_edition, refers to which edition of American Joint Committee on Cancer was used. Not relevant.\
All variables related to age should be removed because of high collinearity with age_at_index, which should not be removed.\
'race' and 'ethnicity' should be removed due to high amount of NaN pr 'not reported' values.\
'days_to_death' cannot be included into a model that aims to predict vital satus.

In [1215]:
df_final2['tumor_descriptor'].unique()

array(['Primary', 'Not Applicable', 'Metastatic', 'Recurrence'],
      dtype=object)

In [1216]:
df_final2['sample_type'].unique()

array(['Primary Tumor', 'Blood Derived Normal', 'Solid Tissue Normal',
       'Metastatic', 'Recurrent Tumor'], dtype=object)

In [1217]:
df_final2['tissue_type'].unique()

array(['Tumor', 'Normal'], dtype=object)

In [1218]:
model_df = df_final2.drop(columns=['age_at_diagnosis','submitter_id','tumor_descriptor','sample_type','tissue_type','ajcc_staging_system_edition','ethnicity','days_to_death','days_to_birth','year_of_birth','days_to_last_follow_up'])

### Missing values

In [1219]:
pd.set_option('display.max_rows', None)
print(model_df.isna().sum())

ajcc_pathologic_stage         24
tissue_or_organ_of_origin      3
primary_diagnosis              3
prior_malignancy               3
ajcc_pathologic_t              5
morphology                     3
ajcc_pathologic_n              5
ajcc_pathologic_m             13
treatment_or_therapy           3
gender                         3
race                           3
vital_status                   3
age_at_index                   4
apc                            0
tp53                           0
kras                           0
muc16                          0
pik3ca                         0
fat4                           0
lrp1b                          0
csmd3                          0
fat3                           0
fbxw7                          0
ptprt                          0
mll4                           0
nbea                           0
arid1a                         0
fam123b                        0
smad4                          0
atm                            0
tcf7l2    

In [1220]:
pd.reset_option('display.max_rows')

Columns like CDH10_exp, DCC_exp and FAM47C_exp have too many missing values. We should remove them.

In [1221]:
model_df = model_df.drop(columns= ['CDH10_exp','DCC_exp','FAM47C_exp'])

Rest of columns have as maximum as 42 missing values, which represents less than 10% of the data, so let's drop those rows

In [1222]:
model_df = model_df.dropna()

### Dummy variables

Let's format some columns\
In 'ajcc_pathologic_stage' column, there is a meaningful ordinal relationship between the stages (Stage I < Stage II < Stage III < Stage IV), however, for some stages like Stage I, we don't know if Stage IA represents a lower stage. For this reasoN we prefer to create dummy variables instead of label encoding.\
Also dummy variables in: 'tissue_or_organ_of_origin', 'primary_diagnosis','morphology', TNM staging, 

In [1223]:
model_df['ajcc_pathologic_stage'].value_counts()


ajcc_pathologic_stage
Stage IIA     167
Stage I       100
Stage IIIB     78
Stage IV       57
Stage IIIC     53
Stage II       34
Stage IVA      24
Stage III      22
Stage IIIA     14
Stage IIB      11
Stage IVB       2
Stage IIC       2
Stage IA        1
Name: count, dtype: int64

In [1224]:
model_df['tissue_or_organ_of_origin'].value_counts()

tissue_or_organ_of_origin
Colon                       103
Sigmoid colon               102
Cecum                        84
Ascending colon              79
Rectum                       77
Rectosigmoid junction        66
Transverse colon             18
Descending colon             16
Hepatic flexure of colon     12
Splenic flexure of colon      5
Connective                    2
Unknown primary site          1
Name: count, dtype: int64

In [1225]:
model_df['primary_diagnosis'].value_counts() 

primary_diagnosis
Adenocarcinoma                                        469
Mucinous adenocarcinoma                                76
Adenocarcinoma in tubolovillous adenoma                 7
Tubular adenocarcinoma                                  5
Adenocarcinoma with mixed subtypes                      3
Papillary adenocarcinoma                                2
Adenocarcinoma with neuroendocrine differentiation      1
Carcinoma                                               1
Adenosquamous carcinoma                                 1
Name: count, dtype: int64

In [1226]:
model_df['morphology'].value_counts()

morphology
8140/3    469
8480/3     76
8263/3      7
8211/3      5
8255/3      3
8260/3      2
8574/3      1
8010/3      1
8560/3      1
Name: count, dtype: int64

In [1227]:
model_df['ajcc_pathologic_t'].value_counts()

ajcc_pathologic_t
T3     385
T2      97
T4      30
T4a     25
T1      18
T4b      9
Tis      1
Name: count, dtype: int64

In [1228]:
model_df['ajcc_pathologic_n'].value_counts()

ajcc_pathologic_n
N0     325
N1      95
N2      76
N1b     20
N1a     18
N2b     17
N2a     11
N1c      3
Name: count, dtype: int64

In [1229]:
model_df['ajcc_pathologic_m'].value_counts()

ajcc_pathologic_m
M0     430
M1      67
MX      54
M1a     11
M1b      3
Name: count, dtype: int64

In [1230]:
model_df = pd.get_dummies(model_df, columns=['ajcc_pathologic_stage','tissue_or_organ_of_origin','primary_diagnosis','morphology','ajcc_pathologic_t','ajcc_pathologic_n','ajcc_pathologic_m'], dtype=int)
print(model_df)

    prior_malignancy treatment_or_therapy  gender                       race  \
0                 no                   no    male                      white   
1                 no                   no  female                      white   
2                 no                   no    male                      white   
3                 no                   no    male               not reported   
4                 no                   no  female               not reported   
..               ...                  ...     ...                        ...   
627               no         not reported    male               not reported   
628               no         not reported  female               not reported   
629               no                  yes  female               not reported   
630               no                   no  female  black or african american   
632               no         not reported    male                      white   

    vital_status  age_at_index  apc  tp

### Label encoding

prior_malignancy, treatment_or_therapy\
gender, race\
vital_status...   

In [1231]:
model_df

,prior_malignancy,treatment_or_therapy,gender,race,vital_status,age_at_index,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,ERBB4_exp,FAM123B_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met,ajcc_pathologic_stage_Stage I,ajcc_pathologic_stage_Stage IA,ajcc_pathologic_stage_Stage II,ajcc_pathologic_stage_Stage IIA,ajcc_pathologic_stage_Stage IIB,ajcc_pathologic_stage_Stage IIC,ajcc_pathologic_stage_Stage III,ajcc_pathologic_stage_Stage IIIA,ajcc_pathologic_stage_Stage IIIB,ajcc_pathologic_stage_Stage IIIC,ajcc_pathologic_stage_Stage IV,ajcc_pathologic_stage_Stage IVA,ajcc_pathologic_stage_Stage IVB,tissue_or_organ_of_origin_Ascending colon,tissue_or_organ_of_origin_Cecum,tissue_or_organ_of_origin_Colon,tissue_or_organ_of_origin_Connective,tissue_or_organ_of_origin_Descending colon,tissue_or_organ_of_origin_Hepatic flexure of colon,tissue_or_organ_of_origin_Rectosigmoid junction,tissue_or_organ_of_origin_Rectum,tissue_or_organ_of_origin_Sigmoid colon,tissue_or_organ_of_origin_Splenic flexure of colon,tissue_or_organ_of_origin_Transverse colon,tissue_or_organ_of_origin_Unknown primary site,primary_diagnosis_Adenocarcinoma,primary_diagnosis_Adenocarcinoma in tubolovillous adenoma,primary_diagnosis_Adenocarcinoma with mixed subtypes,primary_diagnosis_Adenocarcinoma with neuroendocrine differentiation,primary_diagnosis_Adenosquamous carcinoma,primary_diagnosis_Carcinoma,primary_diagnosis_Mucinous adenocarcinoma,primary_diagnosis_Papillary adenocarcinoma,primary_diagnosis_Tubular adenocarcinoma,morphology_8010/3,morphology_8140/3,morphology_8211/3,morphology_8255/3,morphology_8260/3,morphology_8263/3,morphology_8480/3,morphology_8560/3,morphology_8574/3,ajcc_pathologic_t_T1,ajcc_pathologic_t_T2,ajcc_pathologic_t_T3,ajcc_pathologic_t_T4,ajcc_pathologic_t_T4a,ajcc_pathologic_t_T4b,ajcc_pathologic_t_Tis,ajcc_pathologic_n_N0,ajcc_pathologic_n_N1,ajcc_pathologic_n_N1a,ajcc_pathologic_n_N1b,ajcc_pathologic_n_N1c,ajcc_pathologic_n_N2,ajcc_pathologic_n_N2a,ajcc_pathologic_n_N2b,ajcc_pathologic_m_M0,ajcc_pathologic_m_M1,ajcc_pathologic_m_M1a,ajcc_pathologic_m_M1b,ajcc_pathologic_m_MX
0,no,no,male,white,Dead,70.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4924,0.9748,1.3970,0.6288,1.2007,-0.1100,2.0609,0.3974,0.5973,1.8436,1.1990,1.1869,0.0707,-0.0260,-0.7453,0.6173,1.1064,-0.1753,-1.2877,-0.6245,2.0391,-0.1122,2.3358,1.4414,-0.1442,0.0962,-0.4011,-1.1856,1.2024,0.2879,2.1324,-0.0907,0.2257,1.4222,1.6625,0.1280,-0.0253,1.2033,-0.0115,0.2442,-0.0368,-0.3021,0.0596,2.0379,0.5959,1.5926,0.0494,0.081050,0.062878,0.346054,0.319615,0.052065,0.064752,0.049225,0.180710,0.039378,0.029382,0.048825,0.477222,0.612599,0.482879,0.825678,0.409599,0.570405,0.065553,0.422390,0.227364,0.052323,0.059601,0.073225,0.249039,0.037806,0.137725,0.153706,0.034230,0.034982,0.064444,0.434121,0.061711,0.659091,0.109629,0.224915,0.433367,0.058690,0.176453,0.076784,0.125888,0.080165,

In [1232]:
model_df['prior_malignancy'].value_counts()

prior_malignancy
no     513
yes     52
Name: count, dtype: int64

In [1233]:
model_df['treatment_or_therapy'].value_counts()

treatment_or_therapy
no              350
yes             120
not reported     95
Name: count, dtype: int64

In [1234]:
model_df['race'].value_counts()

race
white                               263
not reported                        225
black or african american            64
asian                                12
american indian or alaska native      1
Name: count, dtype: int64

In [1235]:
model_df['gender'].value_counts()

gender
male      297
female    268
Name: count, dtype: int64

In [1236]:
model_df['vital_status'].value_counts()

vital_status
Alive    453
Dead     112
Name: count, dtype: int64

Given the high number of 'not reported' in treatment_or_therapy and race, they should be removed

In [1237]:
model_df = model_df.drop(columns=['treatment_or_therapy','race'])

Let's make numerical prior_malignancy, gender and vital_status

In [1238]:
model_df['prior_malignancy'] = model_df['prior_malignancy'].replace({'no': 0, 'yes': 1})
model_df['gender'] = model_df['gender'].replace({'male': 0, 'female': 1})
model_df['vital_status'] = model_df['vital_status'].replace({'Alive': 0, 'Dead': 1})

C:\Users\juan_\AppData\Local\Temp\ipykernel_21188\4145546519.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  model_df['prior_malignancy'] = model_df['prior_malignancy'].replace({'no': 0, 'yes': 1})
C:\Users\juan_\AppData\Local\Temp\ipykernel_21188\4145546519.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  model_df['gender'] = model_df['gender'].replace({'male': 0, 'female': 1})
C:\Users\juan_\AppData\Local\Temp\ipykernel_21188\4145546519.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remov

In [1239]:
model_df = model_df.drop(columns=['ajcc_pathologic_m_MX'])

In [1240]:
pd.set_option('display.max_rows', None)
pd.DataFrame(model_df.dtypes)


,0
prior_malignancy,int64
gender,int64
vital_status,int64
age_at_index,float64
apc,int64
tp53,int64
kras,int64
muc16,int64
pik3ca,int64
fat4,int64


In [1241]:
pd.reset_option('display.max_rows')

In [1242]:
model_df.to_csv("model_df.csv",  index=False)

In [1243]:
model_df = pd.read_csv("model_df.csv")
model_df

,prior_malignancy,gender,vital_status,age_at_index,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,ERBB4_exp,FAM123B_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met,ajcc_pathologic_stage_Stage I,ajcc_pathologic_stage_Stage IA,ajcc_pathologic_stage_Stage II,ajcc_pathologic_stage_Stage IIA,ajcc_pathologic_stage_Stage IIB,ajcc_pathologic_stage_Stage IIC,ajcc_pathologic_stage_Stage III,ajcc_pathologic_stage_Stage IIIA,ajcc_pathologic_stage_Stage IIIB,ajcc_pathologic_stage_Stage IIIC,ajcc_pathologic_stage_Stage IV,ajcc_pathologic_stage_Stage IVA,ajcc_pathologic_stage_Stage IVB,tissue_or_organ_of_origin_Ascending colon,tissue_or_organ_of_origin_Cecum,tissue_or_organ_of_origin_Colon,tissue_or_organ_of_origin_Connective,tissue_or_organ_of_origin_Descending colon,tissue_or_organ_of_origin_Hepatic flexure of colon,tissue_or_organ_of_origin_Rectosigmoid junction,tissue_or_organ_of_origin_Rectum,tissue_or_organ_of_origin_Sigmoid colon,tissue_or_organ_of_origin_Splenic flexure of colon,tissue_or_organ_of_origin_Transverse colon,tissue_or_organ_of_origin_Unknown primary site,primary_diagnosis_Adenocarcinoma,primary_diagnosis_Adenocarcinoma in tubolovillous adenoma,primary_diagnosis_Adenocarcinoma with mixed subtypes,primary_diagnosis_Adenocarcinoma with neuroendocrine differentiation,primary_diagnosis_Adenosquamous carcinoma,primary_diagnosis_Carcinoma,primary_diagnosis_Mucinous adenocarcinoma,primary_diagnosis_Papillary adenocarcinoma,primary_diagnosis_Tubular adenocarcinoma,morphology_8010/3,morphology_8140/3,morphology_8211/3,morphology_8255/3,morphology_8260/3,morphology_8263/3,morphology_8480/3,morphology_8560/3,morphology_8574/3,ajcc_pathologic_t_T1,ajcc_pathologic_t_T2,ajcc_pathologic_t_T3,ajcc_pathologic_t_T4,ajcc_pathologic_t_T4a,ajcc_pathologic_t_T4b,ajcc_pathologic_t_Tis,ajcc_pathologic_n_N0,ajcc_pathologic_n_N1,ajcc_pathologic_n_N1a,ajcc_pathologic_n_N1b,ajcc_pathologic_n_N1c,ajcc_pathologic_n_N2,ajcc_pathologic_n_N2a,ajcc_pathologic_n_N2b,ajcc_pathologic_m_M0,ajcc_pathologic_m_M1,ajcc_pathologic_m_M1a,ajcc_pathologic_m_M1b
0,0,0,1,70.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4924,0.9748,1.3970,0.6288,1.2007,-0.1100,2.0609,0.3974,0.5973,1.8436,1.1990,1.1869,0.0707,-0.0260,-0.7453,0.6173,1.1064,-0.1753,-1.2877,-0.6245,2.0391,-0.1122,2.3358,1.4414,-0.1442,0.0962,-0.4011,-1.1856,1.2024,0.2879,2.1324,-0.0907,0.2257,1.4222,1.6625,0.1280,-0.0253,1.2033,-0.0115,0.2442,-0.0368,-0.3021,0.0596,2.0379,0.5959,1.5926,0.0494,0.081050,0.062878,0.346054,0.319615,0.052065,0.064752,0.049225,0.180710,0.039378,0.029382,0.048825,0.477222,0.612599,0.482879,0.825678,0.409599,0.570405,0.065553,0.422390,0.227364,0.052323,0.059601,0.073225,0.249039,0.037806,0.137725,0.153706,0.034230,0.034982,0.064444,0.434121,0.061711,0.659091,0.109629,0.224915,0.433367,0.058690,0.176453,0.076784,0.125888,0.080165,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0

## Predictive models for classification

In [1244]:
features = model_df.drop(columns=['vital_status'])
target = model_df['vital_status']

Spliting data into train (80%) and test. 

In [1245]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size= 0.20)

Validation dataset for streamlit

In [1246]:
validation_data = pd.concat([X_test,y_test], axis=1)
validation_data

,prior_malignancy,gender,age_at_index,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,ERBB4_exp,FAM123B_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met,ajcc_pathologic_stage_Stage I,ajcc_pathologic_stage_Stage IA,ajcc_pathologic_stage_Stage II,ajcc_pathologic_stage_Stage IIA,ajcc_pathologic_stage_Stage IIB,ajcc_pathologic_stage_Stage IIC,ajcc_pathologic_stage_Stage III,ajcc_pathologic_stage_Stage IIIA,ajcc_pathologic_stage_Stage IIIB,ajcc_pathologic_stage_Stage IIIC,ajcc_pathologic_stage_Stage IV,ajcc_pathologic_stage_Stage IVA,ajcc_pathologic_stage_Stage IVB,tissue_or_organ_of_origin_Ascending colon,tissue_or_organ_of_origin_Cecum,tissue_or_organ_of_origin_Colon,tissue_or_organ_of_origin_Connective,tissue_or_organ_of_origin_Descending colon,tissue_or_organ_of_origin_Hepatic flexure of colon,tissue_or_organ_of_origin_Rectosigmoid junction,tissue_or_organ_of_origin_Rectum,tissue_or_organ_of_origin_Sigmoid colon,tissue_or_organ_of_origin_Splenic flexure of colon,tissue_or_organ_of_origin_Transverse colon,tissue_or_organ_of_origin_Unknown primary site,primary_diagnosis_Adenocarcinoma,primary_diagnosis_Adenocarcinoma in tubolovillous adenoma,primary_diagnosis_Adenocarcinoma with mixed subtypes,primary_diagnosis_Adenocarcinoma with neuroendocrine differentiation,primary_diagnosis_Adenosquamous carcinoma,primary_diagnosis_Carcinoma,primary_diagnosis_Mucinous adenocarcinoma,primary_diagnosis_Papillary adenocarcinoma,primary_diagnosis_Tubular adenocarcinoma,morphology_8010/3,morphology_8140/3,morphology_8211/3,morphology_8255/3,morphology_8260/3,morphology_8263/3,morphology_8480/3,morphology_8560/3,morphology_8574/3,ajcc_pathologic_t_T1,ajcc_pathologic_t_T2,ajcc_pathologic_t_T3,ajcc_pathologic_t_T4,ajcc_pathologic_t_T4a,ajcc_pathologic_t_T4b,ajcc_pathologic_t_Tis,ajcc_pathologic_n_N0,ajcc_pathologic_n_N1,ajcc_pathologic_n_N1a,ajcc_pathologic_n_N1b,ajcc_pathologic_n_N1c,ajcc_pathologic_n_N2,ajcc_pathologic_n_N2a,ajcc_pathologic_n_N2b,ajcc_pathologic_m_M0,ajcc_pathologic_m_M1,ajcc_pathologic_m_M1a,ajcc_pathologic_m_M1b,vital_status
120,0,1,71.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-0.1978,-0.7243,0.2635,-0.9185,-0.5712,0.1657,-1.3187,1.6445,-0.0765,1.0234,0.0827,0.6106,-1.6360,1.2989,-0.4037,1.8183,0.6723,-1.1649,-0.6773,-1.2580,-2.0461,-0.8062,-1.9922,-2.1266,1.3392,-1.3662,-1.1940,-1.1856,-0.6451,1.0704,3.2839,-1.1198,-0.4342,0.9158,-1.7392,-2.5475,-1.8123,-1.2112,-2.8018,-1.4769,2.5872,-2.2690,1.1977,-0.9590,0.4356,0.1657,-0.3171,0.084119,0.070311,0.149263,0.062937,0.054231,0.404353,0.064541,0.067878,0.461169,0.023690,0.056754,0.480541,0.367667,0.821633,0.850827,0.779119,0.495015,0.043861,0.611539,0.710477,0.152014,0.031596,0.049452,0.487160,0.035719,0.759547,0.179169,0.036002,0.389603,0.044054,0.716166,0.228233,0.632611,0.127416,0.512709,0.172238,0.044765,0.140211,0.070011,0.103016,0.053832,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,

In [1247]:
validation_data.to_csv("C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/validation_data.csv", index = False)

### Logistic regression

In [1248]:
# from sklearn.linear_model import LogisticRegression

# logreg = LogisticRegression()

In [1249]:
# logreg.fit(X_train, y_train)
# y_pred = logreg.predict(X_test)

Let's evaluate our model\
Accuracy = # correct predictions/ #total predictions\
Precision = True positive/ True positive + False positive\
Recall = True positive/ True positive + False negative\
F1-score is the mean of precision and recall.

In [1250]:
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# print(f"RMSE {mean_squared_error(y_pred, y_test, squared=False)}")

# # Additional evaluation metrics
# print(classification_report(y_test, y_pred))

The model predicts quite well class 0 (status alive), but it does not really predicts for class 1.\
The target variable (vital_status) is a bit imbalanced, since in our model dataframe there are aprox four times more alive than dead patients. This explains the lower prediction for class 1 (dead).\
For imbalanced data, KNN and logistic regression algorithms perform poorly. Tree based models improve the ability to deal with imbalanced classes.
So we should try a Random Forest method. But first, let's oversample to see if it solves the problem.

Our data is imbalanced since in our model dataframe there are aprox four times more alive than dead patients. Let's solve that first.

### Oversampling

In [1251]:
from sklearn.utils import resample

train = pd.concat([X_train, y_train], axis = 1)
train.head()

,prior_malignancy,gender,age_at_index,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,ERBB4_exp,FAM123B_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met,ajcc_pathologic_stage_Stage I,ajcc_pathologic_stage_Stage IA,ajcc_pathologic_stage_Stage II,ajcc_pathologic_stage_Stage IIA,ajcc_pathologic_stage_Stage IIB,ajcc_pathologic_stage_Stage IIC,ajcc_pathologic_stage_Stage III,ajcc_pathologic_stage_Stage IIIA,ajcc_pathologic_stage_Stage IIIB,ajcc_pathologic_stage_Stage IIIC,ajcc_pathologic_stage_Stage IV,ajcc_pathologic_stage_Stage IVA,ajcc_pathologic_stage_Stage IVB,tissue_or_organ_of_origin_Ascending colon,tissue_or_organ_of_origin_Cecum,tissue_or_organ_of_origin_Colon,tissue_or_organ_of_origin_Connective,tissue_or_organ_of_origin_Descending colon,tissue_or_organ_of_origin_Hepatic flexure of colon,tissue_or_organ_of_origin_Rectosigmoid junction,tissue_or_organ_of_origin_Rectum,tissue_or_organ_of_origin_Sigmoid colon,tissue_or_organ_of_origin_Splenic flexure of colon,tissue_or_organ_of_origin_Transverse colon,tissue_or_organ_of_origin_Unknown primary site,primary_diagnosis_Adenocarcinoma,primary_diagnosis_Adenocarcinoma in tubolovillous adenoma,primary_diagnosis_Adenocarcinoma with mixed subtypes,primary_diagnosis_Adenocarcinoma with neuroendocrine differentiation,primary_diagnosis_Adenosquamous carcinoma,primary_diagnosis_Carcinoma,primary_diagnosis_Mucinous adenocarcinoma,primary_diagnosis_Papillary adenocarcinoma,primary_diagnosis_Tubular adenocarcinoma,morphology_8010/3,morphology_8140/3,morphology_8211/3,morphology_8255/3,morphology_8260/3,morphology_8263/3,morphology_8480/3,morphology_8560/3,morphology_8574/3,ajcc_pathologic_t_T1,ajcc_pathologic_t_T2,ajcc_pathologic_t_T3,ajcc_pathologic_t_T4,ajcc_pathologic_t_T4a,ajcc_pathologic_t_T4b,ajcc_pathologic_t_Tis,ajcc_pathologic_n_N0,ajcc_pathologic_n_N1,ajcc_pathologic_n_N1a,ajcc_pathologic_n_N1b,ajcc_pathologic_n_N1c,ajcc_pathologic_n_N2,ajcc_pathologic_n_N2a,ajcc_pathologic_n_N2b,ajcc_pathologic_m_M0,ajcc_pathologic_m_M1,ajcc_pathologic_m_M1a,ajcc_pathologic_m_M1b,vital_status
221,0,0,55.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1.6074,-0.0908,-0.4611,-0.6181,-0.3344,0.4016,-0.0278,1.1151,0.4100,0.5486,-1.4116,0.5744,-0.5752,-0.5733,0.1636,0.4101,-0.5078,-1.1649,0.2657,-1.2413,-0.0977,-0.8778,-0.0490,-0.3867,0.4604,0.6845,0.7337,-0.9060,-0.5021,-0.5341,0.2234,-0.7260,-1.9100,-0.2548,-0.4476,0.0262,-1.8123,-0.7150,1.1032,0.1143,3.7549,0.1163,-0.6995,-0.0083,0.4578,-0.1249,-0.1234,0.102059,0.055642,0.202216,0.156333,0.038262,0.024731,0.095686,0.082920,0.024277,0.027243,0.055399,0.484300,0.314219,0.537301,0.818738,0.260205,0.322032,0.226918,0.269464,0.177364,0.035596,0.034756,0.053783,0.209065,0.038955,0.251320,0.128495,0.087797,0.051881,0.048660,0.521579,0.041495,0.305704,0.072800,0.106740,0.253469,0.050557,0.120120,0.078069,0.079822,0.053068,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [1252]:
# separate majority from minority class:

alive  = train[train['vital_status'] == 0]
dead = train[train['vital_status'] == 1]

print(alive.shape)
print(dead.shape)

(358, 204)
(94, 204)


In [1253]:
# instead of 91 dead patients, we want 359 alive patients:

dead_oversampled = resample(dead,
                                    n_samples= len(alive),
                                    replace=True,
                                    random_state=0) # we need replacement in order to be able to reach 370 datapoints, we just dont have enough datapoints

print(alive.shape)
print(dead_oversampled.shape)

(358, 204)
(358, 204)


In [1254]:
train_over = pd.concat([dead_oversampled, alive])
train_over

,prior_malignancy,gender,age_at_index,apc,tp53,kras,muc16,pik3ca,fat4,lrp1b,csmd3,fat3,fbxw7,ptprt,mll4,nbea,arid1a,fam123b,smad4,atm,tcf7l2,braf,zfhx3,robo2,rnf43,rnf213,grin2a,fat1,erbb4,tnc,mll3,acvr2a,trrap,akap9,crebbp,cntnap2,birc6,prex2,atrx,ank1,card11,ctnna2,fam47c,cdh10,bcl9l,myh11,brca2,cdh4,ros1,dcc,bcl9,ptpn13,bcorl1,ACVR2A_exp,AKAP9_exp,ANK1_exp,APC_exp,ARID1A_exp,ATM_exp,ATRX_exp,BCL9_exp,BCL9L_exp,BCORL1_exp,BIRC6_exp,BRAF_exp,BRCA2_exp,CARD11_exp,CDH4_exp,CNTNAP2_exp,CREBBP_exp,CSMD3_exp,CTNNA2_exp,ERBB4_exp,FAM123B_exp,FAT1_exp,FAT3_exp,FAT4_exp,FBXW7_exp,GRIN2A_exp,KRAS_exp,LRP1B_exp,MLL3_exp,MLL4_exp,MUC16_exp,MYH11_exp,NBEA_exp,PIK3CA_exp,PREX2_exp,PTPN13_exp,PTPRT_exp,RNF213_exp,RNF43_exp,ROBO2_exp,ROS1_exp,SMAD4_exp,TCF7L2_exp,TNC_exp,TP53_exp,TRRAP_exp,ZFHX3_exp,ACVR2A_met,AKAP9_met,ANK1_met,APC_met,ARID1A_met,ATRX_met,BCL9_met,BCL9L_met,BCORL1_met,BIRC6_met,BRAF_met,BRCA2_met,CARD11_met,CDH10_met,CDH4_met,CNTNAP2_met,CSMD3_met,CTNNA2_met,DCC_met,ERBB4_met,FAM123B_met,FAT1_met,FBXW7_met,GRIN2A_met,KRAS_met,LRP1B_met,MLL3_met,MLL4_met,MYH11_met,PIK3CA_met,PREX2_met,PTPN13_met,PTPRT_met,RNF213_met,RNF43_met,ROS1_met,SMAD4_met,TCF7L2_met,TNC_met,TRRAP_met,ZFHX3_met,ajcc_pathologic_stage_Stage I,ajcc_pathologic_stage_Stage IA,ajcc_pathologic_stage_Stage II,ajcc_pathologic_stage_Stage IIA,ajcc_pathologic_stage_Stage IIB,ajcc_pathologic_stage_Stage IIC,ajcc_pathologic_stage_Stage III,ajcc_pathologic_stage_Stage IIIA,ajcc_pathologic_stage_Stage IIIB,ajcc_pathologic_stage_Stage IIIC,ajcc_pathologic_stage_Stage IV,ajcc_pathologic_stage_Stage IVA,ajcc_pathologic_stage_Stage IVB,tissue_or_organ_of_origin_Ascending colon,tissue_or_organ_of_origin_Cecum,tissue_or_organ_of_origin_Colon,tissue_or_organ_of_origin_Connective,tissue_or_organ_of_origin_Descending colon,tissue_or_organ_of_origin_Hepatic flexure of colon,tissue_or_organ_of_origin_Rectosigmoid junction,tissue_or_organ_of_origin_Rectum,tissue_or_organ_of_origin_Sigmoid colon,tissue_or_organ_of_origin_Splenic flexure of colon,tissue_or_organ_of_origin_Transverse colon,tissue_or_organ_of_origin_Unknown primary site,primary_diagnosis_Adenocarcinoma,primary_diagnosis_Adenocarcinoma in tubolovillous adenoma,primary_diagnosis_Adenocarcinoma with mixed subtypes,primary_diagnosis_Adenocarcinoma with neuroendocrine differentiation,primary_diagnosis_Adenosquamous carcinoma,primary_diagnosis_Carcinoma,primary_diagnosis_Mucinous adenocarcinoma,primary_diagnosis_Papillary adenocarcinoma,primary_diagnosis_Tubular adenocarcinoma,morphology_8010/3,morphology_8140/3,morphology_8211/3,morphology_8255/3,morphology_8260/3,morphology_8263/3,morphology_8480/3,morphology_8560/3,morphology_8574/3,ajcc_pathologic_t_T1,ajcc_pathologic_t_T2,ajcc_pathologic_t_T3,ajcc_pathologic_t_T4,ajcc_pathologic_t_T4a,ajcc_pathologic_t_T4b,ajcc_pathologic_t_Tis,ajcc_pathologic_n_N0,ajcc_pathologic_n_N1,ajcc_pathologic_n_N1a,ajcc_pathologic_n_N1b,ajcc_pathologic_n_N1c,ajcc_pathologic_n_N2,ajcc_pathologic_n_N2a,ajcc_pathologic_n_N2b,ajcc_pathologic_m_M0,ajcc_pathologic_m_M1,ajcc_pathologic_m_M1a,ajcc_pathologic_m_M1b,vital_status
297,0,1,77.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6603,0.7555,0.0314,0.6067,0.6153,0.0923,0.5267,-0.3685,2.0015,0.0814,0.4057,1.0669,-0.7890,-0.0989,-0.0009,1.0888,0.9158,-1.1649,-0.0296,0.1973,0.5266,-0.0485,2.3240,1.0408,-0.8849,3.2835,0.2127,-1.1856,0.9822,0.2862,-0.8283,2.7665,1.6958,1.1004,2.1508,0.0807,0.3281,1.0760,-1.9546,-0.2760,-0.7502,0.9054,-0.0232,1.7863,-0.5504,0.4503,1.1738,0.052834,0.035956,0.345140,0.479446,0.037731,0.431620,0.049302,0.167847,0.388733,0.018022,0.040957,0.440926,0.688970,0.734331,0.867782,0.535242,0.288698,0.045059,0.499975,0.443699,0.385834,0.076677,0.040969,0.087906,0.032113,0.426236,0.121568,0.037338,0.240456,0.050113,0.642092,0.130390,0.512479,0.140674,0.318534,0.771305,0.047460,0.174613,0.071272,0.096371,0.085389,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0

In [1255]:
X_train_over = train_over.drop("vital_status", axis=1)
y_train_over = train_over["vital_status"]

### Logistic regression after oversampling

In [1256]:
from sklearn.linear_model import LogisticRegression

In [1257]:
logreg = LogisticRegression()

logreg.fit(X_train_over, y_train_over)

y_pred = logreg.predict(X_test)

c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1258]:
y_pred = logreg.predict(X_test)

In [1259]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(f"RMSE {mean_squared_error(y_pred, y_test, squared=False)}")

# Additional evaluation metrics
print(classification_report(y_test, y_pred))

Accuracy: 0.672566371681416
RMSE 0.5722181649673349
              precision    recall  f1-score   support

           0       0.88      0.71      0.78        95
           1       0.24      0.50      0.33        18

    accuracy                           0.67       113
   macro avg       0.56      0.60      0.56       113
weighted avg       0.78      0.67      0.71       113



c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [1260]:
import pickle

with open('C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/logreg.pkl', 'wb') as file:
    pickle.dump(logreg, file)

### KNN

In [1261]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10) 

In [1262]:
knn.fit(X_train_over, y_train_over)

KNeighborsClassifier(n_neighbors=10)

In [1263]:
y_pred = knn.predict(X_test)

In [1264]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)
print(f"RMSE {mean_squared_error(y_pred, y_test, squared=False)}")

# Additional evaluation metrics
print(classification_report(y_test, y_pred))

Accuracy Score: 0.6637168141592921
RMSE 0.5798992893948983
              precision    recall  f1-score   support

           0       0.86      0.72      0.78        95
           1       0.21      0.39      0.27        18

    accuracy                           0.66       113
   macro avg       0.53      0.55      0.53       113
weighted avg       0.76      0.66      0.70       113



c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


To see conclusions look logistic model, more or less same prediction, precision, recall and f1-score

In [1265]:
import pickle

with open('C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/knn.pkl', 'wb') as file:
    pickle.dump(knn, file)

### Random Forest

In [1266]:
from sklearn.ensemble import RandomForestClassifier

In [1267]:
# Instantiate the Random Forest classifier
forest = RandomForestClassifier(n_estimators= 1933,
 min_samples_split = 2,
 min_samples_leaf = 2,
 max_features = 'sqrt',
 max_depth = 110,
 bootstrap = False)

# forest = RandomForestClassifier(n_estimators= 500,
#  max_leaf_nodes = 40,
#  max_depth = 10)

# Train the classifier
forest.fit(X_train_over, y_train_over)

# Make predictions
y_pred = forest.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(f"RMSE {mean_squared_error(y_pred, y_test, squared=False)}")

# Additional evaluation metrics
print(classification_report(y_test, y_pred))

Accuracy: 0.8407079646017699
RMSE 0.3991140631426436
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        95
           1       0.00      0.00      0.00        18

    accuracy                           0.84       113
   macro avg       0.42      0.50      0.46       113
weighted avg       0.71      0.84      0.77       113



c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-package

In [1268]:
import pickle

with open('C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/forest.pkl', 'wb') as file:
    pickle.dump(forest, file)

### Random Search

In [1269]:
# # let us consider a large space of parameters
# # go find parameters to optimize https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 8000, num = 10)]
# # Number of features to consider at every split <- a non-numeric parameter, no problem. Could also have been used in gridsearch
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None) #<- plus no cap
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [1270]:
# # it would be a bit insane to go over all of these combinations
# # random search to the rescue
# from sklearn.model_selection import RandomizedSearchCV

# # in random search you have to say how many iterations you want to get:
# # n_ iter -> how many random combinations he will try

# # n_jobs -> if you want sklearn to parallelize the process. Can also be used in grid search

# random_search = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 10, cv = 5, n_jobs = 10)

In [1271]:
#random_search.fit(X_train_over,y_train_over)

In [1272]:
#random_search.best_params_

In [1273]:
#random_search.score(X_test, y_test)

### Grid research

In [1274]:
# Grid Search goes through all combinations of hyperparameters
# from sklearn.model_selection import GridSearchCV

# we need to define what we consider the "full list" of hyperparameters

# Number of trees in random forest
# n_estimators = [10,100,500,1000]
# # Mximum number of total leaves to consider
# max_leaf_nodes = [15, 30, 40]
# # Maximum number of levels in each tree
# max_depth = [5,10]

In [1275]:
# Create the  grid
# this is a dictionary from hyperparameters to potential values
# the keys in this dictionary have to match the names of the hyperparameters in the documentation of the model
# grid = {'n_estimators': n_estimators,
#         'max_leaf_nodes': max_leaf_nodes,
#         'max_depth': max_depth}

In [1276]:
# # Instantiate the grid search model object

# # estimator -> model to optimize
# forest = RandomForestClassifier()
# # param_grid -> state the dictionary of parameters to optimize
# # cv = 5 -> number of cross validation folds <------ CV is REALLY important in grid search. Why?
# grid_search = GridSearchCV(estimator = forest, param_grid = grid, cv = 5)

In [1277]:
# Fit the grid search to the data
#grid_search.fit(X_train_over, y_train_over)

In [1278]:
## CV is a way to make sure that this best combination
## of characteristics is the best not just because it works well
## for a certain test set but across the board with all your data as test

## ALWAYS ASK YOUR SELVES : IS THIS MODEL REALLY GOOD OR IS HE JUST
## GOOD ON THIS PARTICULAR TEST SET ?

In [1279]:
# and the winner is...
#grid_search.best_params_

In [1280]:
# in grid search you are more likely to get really good results in your training set, even with CV
#grid_search.best_score_

In [1281]:
# and they may not generalize perfectly
#grid_search.score(X_test, y_test)

### XGBoost

In [1282]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size= 0.30)

In [1283]:
import xgboost

xgb_reg = xgboost.XGBClassifier()

xgb_reg.fit(X_train_over, y_train_over)

y_pred = xgb_reg.predict(X_test)
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(f"RMSE {mean_squared_error(y_pred, y_test, squared=False)}")

# Additional evaluation metrics
print(classification_report(y_test, y_pred))

Accuracy: 0.8849557522123894
RMSE 0.3391817326856071
              precision    recall  f1-score   support

           0       0.91      0.96      0.93        95
           1       0.69      0.50      0.58        18

    accuracy                           0.88       113
   macro avg       0.80      0.73      0.76       113
weighted avg       0.88      0.88      0.88       113



c:\Users\juan_\anaconda3\envs\myappenv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [1284]:
import pickle

with open('C:/Users/juan_/OneDrive/Escritorio/Ironhack/Prognostic-Colorectal-cancer-WebApp/xgb_reg.pkl', 'wb') as file:
    pickle.dump(xgb_reg, file)